In [1]:
# requests 패키지 불러오기 (pip install requests 설치 필요)
import requests

# 네이버 영화 사이트 - 탑건: 매버릭 리뷰 페이지 
url = "https://movie.naver.com/movie/bi/mi/review.naver?code=81888"

# requests.get
resp = requests.get(url)
print(resp)

<Response [200]>


In [2]:
# BeautifulSoup 함수 불러오기 (pip install bs4 설치 필요)  
from bs4 import BeautifulSoup 

# BeautifulSoup 함수로, HTML 문서 구조를 파싱 
soup = BeautifulSoup(resp.text, 'html.parser')

# 파싱한 내용을 담고 있는 객체의 자료형 확인
print(type(soup))

<class 'bs4.BeautifulSoup'>


### 1) 영화제목

In [3]:
# <title>탑건: 매버릭 : 네이버 영화</title>

# title 태그 이름을 활용하여 영화 제목이 포함되어 있는 요소를 찾습니다.
title_tag = soup.find(name='title')
print(title_tag)

# 텍스트 부분만 추출합니다.
title_text = title_tag.get_text()
print(title_text)

<title>탑건: 매버릭 : 네이버 영화</title>
탑건: 매버릭 : 네이버 영화


### 2) 리뷰 갯수

In [4]:
# <span class="cnt">총<em>110</em>건</span>

# span 태그의 class 속성값을 활용하여 리뷰 갯수가 포함되어 있는 요소를 찾습니다.
count_tag = soup.find(name='span', attrs={'class':'cnt'})
print("span 태그: ", count_tag)

# count_tag 요소에서 em 태그 부분을 찾습니다. 
count_tag = count_tag.find(name='em')
print("em 태그: ", count_tag)

# 텍스트 부분만 추출합니다.
count_text = count_tag.get_text()
print("텍스트: ", count_text)

span 태그:  <span class="cnt">총<em>107</em>건</span>
em 태그:  <em>107</em>
텍스트:  107


### 3) 리뷰 목록

In [5]:
# <ul class="rvw_list_area">

# ul 태그의 class 속성값을 활용하여 리뷰 제목과 링크가 포함되어 있는 요소를 찾습니다.
review_list_tag = soup.find(name='ul', attrs={'class':'rvw_list_area'})

# review_list_tag 요소에 포함된 li 태그를 모두 찾습니다. 
review_list_tags = review_list_tag.find_all(name='li')
print("li 태그의 수: ", len(review_list_tags))
print("\n")

# 첫 번째 원소를 출력합니다. 
print(review_list_tags[0])

li 태그의 수:  10


<li>
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4808692)"><strong>탑건: 매버릭 (Top Gun: Maverick) 후기와 비하인드</strong></a>
<span class="user"><a href="#" onclick="clickcr(this, 'rli.uid', '', '', event); showReviewListByNid('4808692');">myte****</a> <em>2022.06.14</em><em>추천 6</em></span>
<p><a href="#" onclick="clickcr(this, 'rli.content', '', '', event); showReviewDetail(4808692);">       탑건: 매버릭      감독   조셉 코신스키   출연   톰 크루즈, 제니퍼 코넬리, 마일즈 텔러   개봉   2022.06.22. 미국   평점                        리뷰보기       이 영화를 2주 전에 봤는데 이래저래 결혼식 준비로 바빠서이...</a></p>
</li>


In [6]:
# 첫 번째 아이템에서 리뷰 제목(rli.title), 사용자(rli.uid), 내용(rli.content) 데이터를 추출합니다.

review_title = review_list_tags[0].find_all('a')[0].get_text()
print("제목:", review_title, "\n")

review_uid = review_list_tags[0].find_all('a')[1].get_text()
print("사용자:", review_uid, "\n")

review_content = review_list_tags[0].find_all('a')[2].get_text()
print(r"내용:", review_content, "\n")

제목: 탑건: 매버릭 (Top Gun: Maverick) 후기와 비하인드 

사용자: myte**** 

내용:        탑건: 매버릭      감독   조셉 코신스키   출연   톰 크루즈, 제니퍼 코넬리, 마일즈 텔러   개봉   2022.06.22. 미국   평점                        리뷰보기       이 영화를 2주 전에 봤는데 이래저래 결혼식 준비로 바빠서이... 



In [7]:
# 각 리뷰 페이지로 연결할 nid 값(예: 4808692)을 포함하는 Javascript 코드를 추출합니다.

review_nid = review_list_tags[0].find('a').get('onclick')
review_nid

"clickcr(this, 'rli.title', '', '', event); showReviewDetail(4808692)"

In [8]:
# Javascript 코드에서 7자리 숫자를 추출합니다. (정규식 re 모듈 활용)

import re

review_nid = re.findall('\d{7}', review_nid)[0]
review_nid

'4808692'

In [9]:
# URL을 조합합니다.  
# 예시: https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808973&code=81888&order=#tab

review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code=81888&order=#tab"
review_url

'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808692&code=81888&order=#tab'

In [10]:
# 10개의 리뷰를 반복문으로 조회하면서, 리뷰 제목(rli.title), 사용자(rli.uid), 상세 페이지 url 값을 추출합니다.
title_list = []
uid_list = []
url_list = []

for li_tag in review_list_tags:
    
    review_title = li_tag.find_all('a')[0].get_text()
    title_list.append(review_title)
    
    review_uid = li_tag.find_all('a')[1].get_text()
    uid_list.append(review_uid)
    
    review_nid = re.findall('\d{7}', li_tag.find('a').get('onclick'))[0]
    review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code=81888&order=#tab"
    url_list.append(review_url)    
    
# 추출된 아이템의 수량
print(len(title_list))
print(len(uid_list))
print(len(url_list))

10
10
10


In [11]:
print(title_list)

['탑건: 매버릭 (Top Gun: Maverick) 후기와 비하인드', '탑건:매버릭 ( 전투기가 중요한 것이 아니라 파일럿이 중요하다 )-평점8점', '톰 크루즈의 탑건2가 제작될까요!?', '탑건 매버릭...보고 왔습니다(감상이 자꾸 밀려와서 추가가 자꾸 될거 같습니다)', '[탑건:메버릭] 이제 필요없어 질 수 있는 것들에 대한 위로.', '[탑건: 매버릭]을 보고', '탑건: 매버릭', '[미국 영화] 탑건 : 매버릭', '탑건: 매버릭', '[영화]탑건: 매버릭-롯데시네마 월드타워 수퍼플렉스 G관 A열 체험']


In [12]:
print(uid_list)

['myte****', 'reno****', 'loan****', 'amie****', 'mult****', 'film****', 'them****', 'ford****', 'rome****', 'shoy****']


In [13]:
print(url_list)

['https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808692&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808920&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=3985907&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4809027&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808973&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4809018&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808959&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4810669&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4811078&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808844&code=81888&order=#tab']


### 4) 리뷰 상세 페이지

In [14]:
# 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
resp_text = requests.get(url_list[0])
soup_text = BeautifulSoup(resp_text.text, 'html.parser')

# 리뷰 본문의 텍스트를 추출합니다. /  <div class="user_tx_area"> )
review_text_tag = soup_text.find(name='div', attrs={'class':'user_tx_area'})

# 텍스트 부분만 추출합니다.
review_text = review_text_tag.get_text()
print(review_text)





탑건: 매버릭

감독
조셉 코신스키
출연
톰 크루즈, 제니퍼 코넬리, 마일즈 텔러
개봉
2022.06.22. 미국
평점






리뷰보기

  이 영화를 2주 전에 봤는데 이래저래 결혼식 준비로 바빠서이제와서 쓰는 [탑건: 매버릭] 후기. 정식 개봉 전 부터 평론단 리뷰가 매우 좋아서 기대를 한껏 했었다.고등학생 때 톰 크루즈 및 다른 헐리우드 배우들을 좋아해서그들의 필모 DVD를 싹 모았었는데80년대 끝자락에 태어난 나로서는80년대 영화가 영 익숙하지가 않았던지라 [탑건, 1986]을 본 적이 없었다.탑건 매버릭을 보러가기 이틀 전에 1986년에 개봉한 탑건을 보았다.탑건을 안봐도 무방하겠지만오프닝부터 대놓고 탑건,1986 오마주 장면들이 많아서제대로 된 탑건 후속편을 감상하고 싶으시다면무조건 탑건을 보고 탑건 매버릭을 감상하시는 것을 강력 추천드린다. 탑건 (1986)은 청춘 영화 그 자체였다.그리고 모두가 동의하겠지만 정말로 톰 크루즈 외모가 미쳤다.80년대 탑건이 유명한 이유가 다름아닌 톰 크루즈의 미모 때문이 아니었나 싶을 정도다.당시 분위기는 어떨지 모르겠으나탑건의 각본은 다른 영화와 비교해 크게 다를 게 없었고80년대 특유의 청춘 영화 분위기가 좋았던 영화다.  동영상[탑건: 매버릭] 메인 예고편하늘에서 펼쳐지는 스펙타클 액션✈톰 크루즈의 항공 액션 블록버스터 '탑건: 매버릭'6월 22일, 극장에서 고공 레이스에 합류하라!#탑건_매버릭 #6...www.youtube.com한국은 6월 22일로 개봉이 미뤄진 대신톰 크루즈와 마일즈 텔러, 그리고 글렌 포웰이 내한 프로모션을 돌기로 하였다고 한다.원래 해외 프로모션이라는 개념이 없었는데이런 개념을 만든 장본인이 바로 톰 크루즈라고 한다.심지어 90년대 당시, 한국은 프로모션 대상 국가가 아니었음에도 불구하고톰 크루즈가 처음으로 영화 프로모션으로 한국에 방문했다고 하니'친절한 톰 아저씨' 닉값 톡톡히 하시는듯.36년 만에 돌아온 탑건 매버릭은 현시대를 잘 반영한 영화다.80년대 인터넷, 컴퓨터도

### 5) 10개 리뷰 본문을 모두 수집 -> List로 정리

In [15]:
import time
# url 반복하여 텍스트를 추출하고 리스트에 추가
text_list = []

for url in url_list:

    # 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
    resp_text = requests.get(url)
    soup_text = BeautifulSoup(resp_text.text, 'html.parser')

    # 리뷰 본문의 텍스트를 추출합니다. /  <div class="user_tx_area"> )
    review_text_tag = soup_text.find(name='div', attrs={'class':'user_tx_area'})

    # 텍스트 부분만 추출합니다.
    review_text = review_text_tag.get_text()
    text_list.append(review_text)
    
    # 대기 시간을 추가합니다. (서버에 과도한 호출이 되지 않도록 유의) 
    time.sleep(2)
    
    
# 추출된 아이템의 수량
print(len(text_list))

10


### 6) 판다스 데이터 프레임으로 정리하고 csv 파일로 저장

In [16]:
# 딕셔너리 형식으로 항목별 리스트를 원소로 추가 
dict_data = {
    'title' : title_list,
    'user' : uid_list,
    'review' : text_list   
}

# 판다스 데이터프레임으로 변환
import pandas as pd

df_data = pd.DataFrame(dict_data)

# 변환 결과를 확인
df_data

,title,user,review
0,탑건: 매버릭 (Top Gun: Maverick) 후기와 비하인드,myte****,"\n\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 제니퍼..."
1,탑건:매버릭 ( 전투기가 중요한 것이 아니라 파일럿이 중요하다 )-평점8점,reno****,"\n\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 제니퍼..."
2,톰 크루즈의 탑건2가 제작될까요!?,loan****,\n톰 크루즈의 탑건2가 제작될까요!?\n \n \n \n \n벌써 50이 훌쩍 넘...
3,탑건 매버릭...보고 왔습니다(감상이 자꾸 밀려와서 추가가 자꾸 될거 같습니다),amie****,"\n\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 마일즈..."
4,[탑건:메버릭] 이제 필요없어 질 수 있는 것들에 대한 위로.,mult****,"\n\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 마일즈..."
5,[탑건: 매버릭]을 보고,film****,"\n\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 제니퍼..."
6,탑건: 매버릭,them****,"\n\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 제니퍼..."
7,[미국 영화] 탑건 : 매버릭,ford****,"\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 마일즈 텔..."
8,탑건: 매버릭,rome****,"\n영화는 이렇게 만들어야지..쓸데없는 철학도 빼고, 주장도 빼고.간단 명료한 스토..."
9,[영화]탑건: 매버릭-롯데시네마 월드타워 수퍼플렉스 G관 A열 체험,shoy****,"\n\n\n\n탑건: 매버릭\n\n감독\n조셉 코신스키\n출연\n톰 크루즈, 제니퍼..."


In [17]:
# CSV 파일로 저장
df_data.to_csv("naver_review.csv")